<a href="https://colab.research.google.com/github/TrinhDinhPhuc/100DaysCoding-Algoexpert/blob/master/Compound_Classification_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!curl -Lo conda_installer.py https://raw.githubusercontent.com/deepchem/deepchem/master/scripts/colab_install.py
import conda_installer
conda_installer.install()
!/root/miniconda/bin/conda info -e

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  3490  100  3490    0     0  36354      0 --:--:-- --:--:-- --:--:-- 36354


add /root/miniconda/lib/python3.6/site-packages to PYTHONPATH
python version: 3.6.9
fetching installer from https://repo.continuum.io/miniconda/Miniconda3-latest-Linux-x86_64.sh
done
installing miniconda to /root/miniconda
done
installing rdkit, openmm, pdbfixer
added conda-forge to channels
added omnia to channels
done
conda packages installation finished!


# conda environments:
#
base                  *  /root/miniconda



In [2]:
!pip install --pre deepchem

     |████████████████████████████████| 552kB 17.4MB/s 


In [3]:
import deepchem as dc
from rdkit import Chem
import numpy as np
import pandas as pd
import rdkit.Chem as Chem
import rdkit.Chem.AllChem as AllChem
from sklearn.ensemble import RandomForestClassifier
import sklearn.metrics as metrics

In [4]:
!wget https://filebin.net/962kr79d2qunjf2f/cmpd.csv?t=xe3dll6t

--2021-02-19 05:35:35--  https://filebin.net/962kr79d2qunjf2f/cmpd.csv?t=xe3dll6t
Resolving filebin.net (filebin.net)... 185.47.40.36, 2a02:c0:2f0:700:f816:3eff:fe73:c194
Connecting to filebin.net (filebin.net)|185.47.40.36|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: unspecified [text/html]
Saving to: ‘cmpd.csv?t=xe3dll6t’

cmpd.csv?t=xe3dll6t     [ <=>                ]   3.28K  --.-KB/s    in 0s      

2021-02-19 05:35:35 (77.6 MB/s) - ‘cmpd.csv?t=xe3dll6t’ saved [3363]



In [ ]:
!mv cmpd.csv?t=xe3dll6t cmpd.csv

In [ ]:
cmpd_df = pd.read_csv('cmpd.csv')
cmpd_df.head()

ParserError: ignored

In [ ]:
cmpd_df["label"] = None
for row in cmpd_df.itertuples():
  if row.activity == "active":
    cmpd_df.at[row.Index,'label'] = 1
  else:
    cmpd_df.at[row.Index,'label'] = 0

In [ ]:
cmpd_df["smileToMol"] = None

import deepchem as dc
from rdkit import Chem
def smileToMol(smile):
  # smile = 'CNC(=O)c1cc(Oc2ccc(NC(=O)Nc3ccc(Cl)c(C(F)(F)F)c3)c(F)c2)ccn1'
  molecules = []
  molecules.append(Chem.MolFromSmiles(smile))
  featurizer = dc.feat.graph_features.ConvMolFeaturizer()
  mol_object = featurizer.featurize(molecules)
  return mol_object

for row in cmpd_df.itertuples():
  cmpd_df.at[row.Index,'smileToMol'] = smileToMol(row.smiles)
cmpd_df.head()

,inchikey,smiles,group,activity,label,smileToMol
0,FNHKPVJBJVTLMP-UHFFFAOYSA-N,CNC(=O)c1cc(Oc2ccc(NC(=O)Nc3ccc(Cl)c(C(F)(F)F)...,train,active,1,[<deepchem.feat.mol_graphs.ConvMol object at 0...
1,CUDVHEFYRIWYQD-UHFFFAOYSA-N,CNC(=O)c1cccc2cc(Oc3ccnc4cc(OCC5(N)CC5)c(OC)cc...,train,active,1,[<deepchem.feat.mol_graphs.ConvMol object at 0...
2,TTZSNFLLYPYKIL-UHFFFAOYSA-N,Cc1cc2cc(Oc3ccnc(Nc4cccc(CS(=O)(=O)NCCN(C)C)c4...,test,active,1,[<deepchem.feat.mol_graphs.ConvMol object at 0...
3,UOVCGJXDGOGOCZ-UHFFFAOYSA-N,COc1cc2c(cc1F)C(c1ccccc1Cl)=Nc1c(n[nH]c1C)N2,train,active,1,[<deepchem.feat.mol_graphs.ConvMol object at 0...
4,CUIHSIWYWATEQL-UHFFFAOYSA-N,Cc1ccc(Nc2nccc(N(C)c3ccc4c(C)n(C)nc4c3)n2)cc1S...,test,active,1,[<deepchem.feat.mol_graphs.ConvMol object at 0...


In [ ]:
data_train = cmpd_df[cmpd_df.group == "train"]
data_train.shape

(3977, 6)

In [ ]:
data_train.to_csv("data_train.csv", index=False)

In [ ]:
data_test = cmpd_df[cmpd_df.group == "test"]
data_test.shape

(1553, 6)

In [ ]:
data_test.to_csv("data_test.csv", index=False)

In [8]:
import os
import deepchem as dc
from deepchem.molnet.load_function.molnet_loader import TransformerGenerator, _MolnetLoader
from deepchem.data import Dataset
from typing import List, Optional, Tuple, Union

TASKS = [
    'active', 'inactive'
]

class _Loader(_MolnetLoader):
  def create_dataset(self) -> Dataset:
    print(os.path.join(self.data_dir))
    dataset_file = os.path.join(self.data_dir, "data.csv.gz")
    if not os.path.exists(dataset_file):
      print("the file does not exist !!!")
    print("self.tasks = ")
    print(self.tasks)
    loader = dc.data.CSVLoader(
        tasks=self.tasks, feature_field="smiles", featurizer=self.featurizer)
    return loader.create_dataset(dataset_file, shard_size=8192)

def load_dataset(
    featurizer: Union[dc.feat.Featurizer, str] = 'ECFP',
    splitter: Union[dc.splits.Splitter, str, None] = 'scaffold',
    transformers: List[Union[TransformerGenerator, str]] = ['balancing'],
    reload: bool = True,
    data_dir: Optional[str] = None,
    save_dir: Optional[str] = None,
    **kwargs

) -> Tuple[List[str], Tuple[Dataset, ...], List[dc.trans.Transformer]]:
  print("oh yes")
  loader = _Loader(featurizer, splitter, transformers, TASKS,
                        data_dir, save_dir, **kwargs)
  return loader.load_dataset('data_train', reload)

In [9]:
ls /tmp

dap_multiplexer.072136a36fc9.root.log.INFO.20210219-053136.52
dap_multiplexer.INFO@
data.csv.gz
debugger_1yipd2ivei=
initgoogle_syslog_dir.0/
tmpefwtd2sp/


In [ ]:
!rm -rf /tmp/tox21.csv.gz
!rm -rf /tmp/tox21-featurized

In [ ]:
ls /tmp/tox21-featurized/ConvMolFeaturizer/ScaffoldSplitter/BalancingTransformer

ls: cannot access '/tmp/tox21-featurized/ConvMolFeaturizer/ScaffoldSplitter/BalancingTransformer': No such file or directory


In [10]:
import deepchem as dc
tasks, datasets, transformers = load_dataset(featurizer='GraphConv')

train_dataset, valid_dataset, test_dataset = datasets

oh yes
/tmp
self.tasks = 
['active', 'inactive']


In [11]:
train_dataset

<DiskDataset X.shape: (4424,), y.shape: (4424, 2), w.shape: (4424, 2), task_names: ['active' 'inactive']>

In [51]:
from time import time
import os

n_tasks = len(tasks)
model = dc.models.GraphConvModel(n_tasks, mode='classification', tensorboard=True)
logdir = os.path.join("logs", str(time()))
tensorboard_callback = tf.keras.callbacks.TensorBoard(logdir, histogram_freq=1)
model.fit(train_dataset, nb_epoch=50, callbacks=[tensorboard_callback])

/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/indexed_slices.py:437: UserWarning: Converting sparse IndexedSlices(IndexedSlices(indices=Tensor("gradient_tape/private__graph_conv_keras_model_6/graph_pool_17/Reshape_14:0", shape=(412,), dtype=int32), values=Tensor("gradient_tape/private__graph_conv_keras_model_6/graph_pool_17/Reshape_13:0", shape=(412, 64), dtype=float32), dense_shape=Tensor("gradient_tape/private__graph_conv_keras_model_6/graph_pool_17/Cast_4:0", shape=(2,), dtype=int32))) to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "shape. This may consume a large amount of memory." % value)
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/indexed_slices.py:437: UserWarning: Converting sparse IndexedSlices(IndexedSlices(indices=Tensor("gradient_tape/private__graph_conv_keras_model_6/graph_pool_17/Reshape_17:0", shape=(3176,), dtype=int32), values=Tensor("gradient_tape/private__graph_conv_keras_model_6/graph_poo

TypeError: ignored

In [40]:
!tensorboard --logdir=model.model_dir

2021-02-19 06:23:05.882749: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.10.1
Serving TensorBoard on localhost; to expose to the network, use a proxy or pass --bind_all
TensorBoard 2.4.1 at http://localhost:6006/ (Press CTRL+C to quit)


In [43]:
model.model_dir

'/tmp/tmpksy4xvub'

In [45]:
%load_ext tensorboard.notebook
tensorboard_callback = tf.keras.callbacks.TensorBoard(model.model_dir, histogram_freq=1)

%tensorboard — logdir logs

ERROR: Failed to launch TensorBoard (exited with 2).
Contents of stderr:
2021-02-19 06:27:21.294390: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.10.1
usage: tensorboard [-h] [--helpfull] {serve,dev} ...
tensorboard: error: invalid choice: '—' (choose from 'serve', 'dev')

In [13]:
len(tasks)

2

In [14]:
metric = dc.metrics.Metric(dc.metrics.roc_auc_score)
print('Training set score:', model.evaluate(train_dataset, [metric], transformers))
print('Test set score:', model.evaluate(test_dataset, [metric], transformers))

Training set score: {'roc_auc_score': 0.9933006158440094}
Test set score: {'roc_auc_score': 0.8535096909376638}


In [21]:
from deepchem.models.layers import GraphConv, GraphPool, GraphGather
import tensorflow as tf
import tensorflow.keras.layers as layers

batch_size = 100

class MyGraphConvModel(tf.keras.Model):

  def __init__(self):
    super(MyGraphConvModel, self).__init__()
    self.gc1 = GraphConv(128, activation_fn=tf.nn.tanh)
    self.batch_norm1 = layers.BatchNormalization()
    self.gp1 = GraphPool()

    self.gc2 = GraphConv(128, activation_fn=tf.nn.tanh)
    self.batch_norm2 = layers.BatchNormalization()
    self.gp2 = GraphPool()

    self.dense1 = layers.Dense(256, activation=tf.nn.tanh)
    self.batch_norm3 = layers.BatchNormalization()
    self.readout = GraphGather(batch_size=batch_size, activation_fn=tf.nn.tanh)

    self.dense2 = layers.Dense(n_tasks*2)
    self.logits = layers.Reshape((n_tasks, 2))
    self.softmax = layers.Softmax()

  def call(self, inputs):
    gc1_output = self.gc1(inputs)
    batch_norm1_output = self.batch_norm1(gc1_output)
    gp1_output = self.gp1([batch_norm1_output] + inputs[1:])

    gc2_output = self.gc2([gp1_output] + inputs[1:])
    batch_norm2_output = self.batch_norm1(gc2_output)
    gp2_output = self.gp2([batch_norm2_output] + inputs[1:])

    dense1_output = self.dense1(gp2_output)
    batch_norm3_output = self.batch_norm3(dense1_output)
    readout_output = self.readout([batch_norm3_output] + inputs[1:])

    logits_output = self.logits(self.dense2(readout_output))
    return self.softmax(logits_output)

In [22]:
model = dc.models.KerasModel(
    MyGraphConvModel(), 
    loss=dc.models.losses.CategoricalCrossEntropy())

In [17]:
for ind, (X_b, y_b, w_b, ids_b) in enumerate(train_dataset.iterbatches(deterministic=False, pad_batches=True)):
  print(ind, (X_b, y_b, w_b, ids_b))
  print("X_b", X_b)
  print("X_b.shape= ", X_b.shape)
  print("y_b", y_b)
  print("y_b.shape= ", y_b.shape)
  print("w_b", w_b)
  print("w_b.shape=", w_b.shape)
  print("ids_b", ids_b)
  break

0 (array([<deepchem.feat.mol_graphs.ConvMol object at 0x7fb2a2ebd6a0>,
       <deepchem.feat.mol_graphs.ConvMol object at 0x7fb2a5286438>, ...,
      dtype=object), array([[1., 0.],
       [0., 1.],
       [1., 0.],
       ...,
       [0., 1.],
       [0., 1.],
       [0., 1.]]), array([[1.79691308, 1.79691308],
       [2.254842  , 2.254842  ],
       [1.79691308, 1.79691308],
       ...,
       [2.254842  , 2.254842  ],
       [2.254842  , 2.254842  ],
       [2.254842  , 2.254842  ]]), array(['COc1cc(OC)cc(N(c2ccc3ncc(-c4cnn(C)c4)nc3c2)C(C)(C)C#Cc2nccc(OC)n2)c1',
       'Cc1cccc(NC(=O)Nc2ccc(-c3ccnc4[nH]nc(C)c34)cc2)c1',
       'COc1cc(C)cc2cc(-c3c(CN4CCNC(=O)C4)c(C)n4ncnc(N)c34)sc12', ...,
       'COc1ccc(Nc2cc(NC(=O)/C=C/c3ccccc3)ncn2)cc1',
       'CCOc1ccccc1C=NNC(=O)c1nnn(-c2nonc2N)c1N1CCCC1',
       'COC(=O)Nc1nc2cc(Oc3ccc(NC(=O)Nc4cccc(Br)c4)cc3)ccc2[nH]1.Cl'],
      dtype=object))
X_b [<deepchem.feat.mol_graphs.ConvMol object at 0x7fb2a2ebd6a0>
 <deepchem.feat.mol_graphs.ConvM

In [23]:
from deepchem.metrics import to_one_hot
from deepchem.feat.mol_graphs import ConvMol
import numpy as np

def data_generator(dataset, epochs=1):
  for ind, (X_b, y_b, w_b, ids_b) in enumerate(dataset.iterbatches(batch_size, epochs,
                                                                   deterministic=False, pad_batches=True)):
    multiConvMol = ConvMol.agglomerate_mols(X_b)
    inputs = [multiConvMol.get_atom_features(), multiConvMol.deg_slice, np.array(multiConvMol.membership)]
    for i in range(1, len(multiConvMol.get_deg_adjacency_lists())):
      inputs.append(multiConvMol.get_deg_adjacency_lists()[i])
    labels = [to_one_hot(y_b.flatten(), 2).reshape(-1, n_tasks, 2)]
    weights = [w_b]
    yield (inputs, labels, weights)

In [30]:
model.get_params()

/usr/local/lib/python3.6/dist-packages/sklearn/base.py:197: FutureWarning: From version 0.24, get_params will raise an AttributeError if a parameter cannot be retrieved as an instance attribute. Previously it would return None.
  FutureWarning)


{'batch_size': 100,
 'learning_rate': None,
 'log_frequency': 100,
 'loss': None,
 'model': <__main__.MyGraphConvModel at 0x7fb2a3b427f0>,
 'model_dir': '/tmp/tmpnal0w9lj',
 'optimizer': <deepchem.models.optimizers.Adam at 0x7fb2a2ae2fd0>,
 'output_types': None,
 'tensorboard': False,
 'wandb': False}

In [24]:
model.fit_generator(data_generator(train_dataset, epochs=50))

ValueError: ignored

In [25]:
print('Training set score:', model.evaluate_generator(data_generator(train_dataset), [metric], transformers))
print('Test set score:', model.evaluate_generator(data_generator(test_dataset), [metric], transformers))

ValueError: ignored